# Run CoCiP over a flight

This tutorial walks through an example of running the [Contrail Cirrus Predicition (CoCiP)](https://gmd.copernicus.org/articles/5/543/2012/) model evaluation along a flight trajectory.

## References

- Schumann, U. “A Contrail Cirrus Prediction Model.” Geoscientific Model Development 5, no. 3 (May 3, 2012): 543–80. https://doi.org/10.5194/gmd-5-543-2012.
- Schumann, U., B. Mayer, K. Graf, and H. Mannstein. “A Parametric Radiative Forcing Model for Contrail Cirrus.” Journal of Applied Meteorology and Climatology 51, no. 7 (July 2012): 1391–1406. https://doi.org/10.1175/JAMC-D-11-0242.1.
- Teoh, Roger, Ulrich Schumann, Arnab Majumdar, and Marc E. J. Stettler. “Mitigating the Climate Forcing of Aircraft Contrails by Small-Scale Diversions and Technology Adoption.” Environmental Science & Technology 54, no. 5 (March 3, 2020): 2941–50. https://doi.org/10.1021/acs.est.9b05608.
- Teoh, Roger, Ulrich Schumann, Edward Gryspeerdt, Marc Shapiro, Jarlath Molloy, George Koudis, Christiane Voigt, and Marc Stettler. “Aviation Contrail Climate Effects in the North Atlantic from 2016&ndash;2021.” Atmospheric Chemistry and Physics Discussions, March 30, 2022, 1–27. https://doi.org/10.5194/acp-2022-169.

In [30]:
import pandas as pd

from pycontrails import Flight
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.cocip import Cocip
from pycontrails.models.humidity_scaling import ConstantHumidityScaling
from pycontrails.physics import units

## Load Flight

Load flight trajectory from dataset prepared by Roger Teoh in https://doi.org/10.5194/acp-2022-169

In [31]:
# load flight waypoints
df_flight = pd.read_csv("/content/sample_data/flight-cocip.csv")
df_flight.head()

,Longitude (degrees),Latitude (degrees),Altitude (feet),UTC time,True airspeed (m s-1),Mach Number,Aircraft mass (kg),Fuel mass flow rate (kg s-1),Overall propulsion efficiency,nvPM number emissions index (kg-1),ICAO Aircraft Type,Wingspan (m)
0,-10.070,55.185,36000,1546651185,230.858,0.791,236479.000,1.654,0.4,1500000000000000,A359,64.75
1,-10.273,55.222,36000,1546651245,230.682,0.790,236379.755,1.657,0.4,1500000000000000,A359,64.75
2,-10.476,55.258,36000,1546651305,230.563,0.789,236280.355,1.659,0.4,1500000000000000,A359,64.75
3,-10.680,55.295,36000,1546651365,230.501,0.789,236180.791,1.661,0.4,1500000000000000,A359,64.75
4,-10.883,55.331,36000,1546651425,230.476,0.789,236081.128,1.662,0.4,1500000000000000,A359,64.75


In [32]:
# constant properties along the length of the flight
attrs = {
    "flight_id": "fid",
    "aircraft_type": df_flight["ICAO Aircraft Type"].values[0],
    "wingspan": df_flight["Wingspan (m)"].values[0],
}

Process the flight into a format expected by `pycontrails`. See [pycontrails.Flight](https://py.contrails.org/api/pycontrails.Flight.html#pycontrails.Flight) for interface details.

In [33]:
# convert UTC timestamp to np.datetime64
df_flight["time"] = pd.to_datetime(df_flight["UTC time"], origin="unix", unit="s")

# set altitude in m
df_flight["altitude"] = units.ft_to_m(df_flight["Altitude (feet)"])

# rename a few columns for compatibility with `Flight` requirements
df_flight = df_flight.rename(
    columns={
        "Longitude (degrees)": "longitude",
        "Latitude (degrees)": "latitude",
        "True airspeed (m s-1)": "true_airspeed",
        "Mach Number": "mach_number",
        "Aircraft mass (kg)": "aircraft_mass",
        "Fuel mass flow rate (kg s-1)": "fuel_flow",
        "Overall propulsion efficiency": "engine_efficiency",
        "nvPM number emissions index (kg-1)": "nvpm_ei_n",
    }
)

# clean up a few columns before building Flight class
df_flight = df_flight.drop(
    columns=["ICAO Aircraft Type", "Wingspan (m)", "UTC time", "Altitude (feet)"]
)

fl = Flight(data=df_flight, attrs=attrs)
fl

Flight [10 keys x 162 length, 3 attributes]
	Keys: longitude, latitude, true_airspeed, mach_number, aircraft_mass, ..., time
	Attributes:
	time                [2019-01-05 01:19:45, 2019-01-05 04:00:21]
	longitude           [-50.0, -10.07]
	latitude            [55.185, 61.089]
	altitude            [10972.8, 10972.8]
	flight_id           fid
	aircraft_type       A359
	wingspan            64.75

## Load meteorology from ECMWF

In [34]:
# get met domain from Flight
time = (
    pd.to_datetime(fl["time"][0]).floor("H"),
    pd.to_datetime(fl["time"][-1]).ceil("H") + pd.Timedelta("10H"),
)

# select pressure levels
pressure_levels = [
    400,
    350,
    300,
    250,
    225,
    200,
    175,
    150,
]

<ipython-input-34-703c643b8a43>:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.to_datetime(fl["time"][0]).floor("H"),
<ipython-input-34-703c643b8a43>:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.to_datetime(fl["time"][-1]).ceil("H") + pd.Timedelta("10H"),
<ipython-input-34-703c643b8a43>:4: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  pd.to_datetime(fl["time"][-1]).ceil("H") + pd.Timedelta("10H"),


In [44]:
# downloads met data from CDS
era5pl = ERA5(time=time, variables=Cocip.met_variables, pressure_levels=pressure_levels)
era5sl = ERA5(
    time=time,
    variables=Cocip.rad_variables,
)
time = ("2022-03-01 00:00:00", "2022-03-01 03:00:00")
variables = ["t", "q", "u", "v", "w", "ciwc", "z", "cc"]
pressure_levels = [300, 250, 200]

In [53]:
!pip install cdsapi

In [54]:
import cdsapi
c = cdsapi.Client()
c.retrieve(
    'reanalysis-era5-land',
    {
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
        ],
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
            '1987', '1988', '1989',
            '1990', '1991', '1992',
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016',
            '2017', '2018', '2019',
            '2020',
        ],
        'month': '12',
        'day': '15',
        'time': '12:00',
        'format': 'netcdf',
        'area': [
            60, -10, 35,
            30,
        ],
    },
    './data/era5-land_eur_1981_2020.nc')

HTTPError: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-land


In [51]:
import cdsapi

client = cdsapi.Client()

dataset = 'reanalysis-era5-pressure-levels'
request = {
    'product_type': ['reanalysis'],
    'variable': ['geopotential'],
    'year': ['2024'],
    'month': ['03'],
    'day': ['01'],
    'time': ['13:00'],
    'pressure_level': ['1000'],
    'data_format': 'grib',
}
target = 'download.grib'

client.retrieve(dataset, request, target)

HTTPError: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-pressure-levels


In [55]:
from pycontrails.datalib.ecmwf import ERA5

time = ("2022-03-01 00:00:00", "2022-03-01 23:00:00")
pressure_levels = [350, 300, 250, 225, 200, 175, 150]
met_variables = ["t", "q", "u", "v", "w", "ciwc", "z", "cc"]
rad_variables = ["tsr", "ttr"]

ERA5(time=time, variables=met_variables, pressure_levels=pressure_levels).open_metdataset()
ERA5(time=time, variables=rad_variables).open_metdataset()

HTTPError: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-pressure-levels


In [56]:
# create `MetDataset` from sources
met = era5pl.open_metdataset()
rad = era5sl.open_metdataset()

HTTPError: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-pressure-levels


## Set up model

In [ ]:
params = {
    "process_emissions": False,
    "verbose_outputs": True,
    "humidity_scaling": ConstantHumidityScaling(rhi_adj=0.98),
}
cocip = Cocip(met=met, rad=rad, params=params)

## Run model

In [ ]:
fl_out = cocip.eval(source=fl)

## Review output

The output flight has the original flight data with many new variables added from the evaluation.

In [ ]:
fl_out

In [ ]:
fl_out.dataframe.columns

The `cocip` variable describes where persistent contrails form. It can take on values:

- 1: Persistent contrails form
- 0: No persistent contrails form

In [ ]:
fl_out["cocip"]

The model class contains information about the contrail created:

- `cocip.source` the original input flight
- `cocip.contrail` will be defined as a `pandas` DataFrame if a contrail is created.
- `cocip.contrail_dataset` is the same data but formatted as an `xarray` Dataset.

In [ ]:
cocip.contrail

We can visualize the contrail on top of the original flight trajectory using pandas plotting capabilities

In [ ]:
ax = cocip.source.dataframe.plot(
    "longitude", "latitude", color="k", label=fl.attrs["flight_id"], figsize=(12, 8)
)
cocip.contrail.plot.scatter("longitude", "latitude", c="rf_lw", cmap="Reds", ax=ax);

In [ ]:
ax = cocip.source.dataframe.plot(
    "longitude", "latitude", color="k", label=fl.attrs["flight_id"], figsize=(12, 8)
)
cocip.contrail.plot.scatter(
    "longitude", "latitude", c="ef", cmap="coolwarm", vmin=-1e12, vmax=1e12, ax=ax
);